In [48]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling
import os
import xgboost as xgb

C:\Users\kkksk\AppData\Local\Temp\ipykernel_21356\1402710945.py:4: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


In [16]:
cd("C:/Users/kkksk/Development/Kaggle-Parkinson/amp-parkinsons-disease-progression-prediction")


[WinError 123] 파일 이름, 디렉터리 이름 또는 볼륨 레이블 구문이 잘못되었습니다: '("C:/Users/kkksk/Development/Kaggle-Parkinson/amp-parkinsons-disease-progression-prediction")'
C:\Users\kkksk\Development\Kaggle-Parkinson\amp-parkinsons-disease-progression-prediction\example_test_files


In [34]:
import pandas as pd

train_clinical_data = pd.read_csv("train_clinical_data.csv")
train_peptides = pd.read_csv('train_peptides.csv')
train_proteins = pd.read_csv('train_proteins.csv')
supplemental_clinical_data = pd.read_csv("supplemental_clinical_data.csv")


In [26]:
print(f"Train Clinical Dataframe shape has {train_clinical_data.shape[0]} rows and {train_clinical_data.shape[1]} columns")
train_clinical_data.head()

Train Clinical Dataframe shape has 2615 rows and 8 columns


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


In [25]:
print(f"Train Peptides Dataframe shape has {train_peptides.shape[0]} rows and {train_peptides.shape[1]} columns")
train_peptides.head()

Train Peptides Dataframe shape has 981834 rows and 6 columns


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [27]:
print(f"Train Proteins Dataframe shape has {train_proteins.shape[0]} rows and {train_proteins.shape[1]} columns")
train_proteins.head()

Train Proteins Dataframe shape has 232741 rows and 5 columns


,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [35]:
print(f"Supplemental Clinical Dataframe shape has {supplemental_clinical_data.shape[0]} rows and {supplemental_clinical_data.shape[1]} columns")
supplemental_clinical_data.head()

Supplemental Clinical Dataframe shape has 2223 rows and 8 columns


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,35_0,35,0,5.0,3.0,16.0,0.0,NaN
1,35_36,35,36,6.0,4.0,20.0,0.0,NaN
2,75_0,75,0,4.0,6.0,26.0,0.0,NaN
3,75_36,75,36,1.0,8.0,38.0,0.0,On
4,155_0,155,0,NaN,NaN,0.0,NaN,NaN


In [36]:
from sklearn.preprocessing import LabelEncoder

train_proteins.loc[:, "UniProt"] = train_proteins.loc[:, "UniProt"].fillna("None")
train_proteins.loc[:, "NPX"] = train_proteins.loc[:, "NPX"].fillna(0)
le = LabelEncoder()
train_proteins['UniProt'] = le.fit_transform(train_proteins['UniProt'])
train_proteins.head()

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,0,11254.3
1,55_0,0,55,1,732430.0
2,55_0,0,55,2,39585.8
3,55_0,0,55,3,41526.9
4,55_0,0,55,4,31238.0


In [37]:
protein_aux_1 = pd.DataFrame(train_proteins.groupby(by=["patient_id", "visit_id", "visit_month"])["UniProt"].apply(list))#.reset_index()
protein_aux_2 = pd.DataFrame(train_proteins.groupby(by=["patient_id", "visit_id", "visit_month"])["NPX"].apply(list))#.reset_index()
protein_aux_3 = pd.DataFrame(protein_aux_1.UniProt.values.tolist()).add_prefix('UniProt_')
protein_aux_4 = pd.DataFrame(protein_aux_2.NPX.values.tolist()).add_prefix('NPX_')
protein_aux_3.index = protein_aux_1.index
protein_aux_4.index = protein_aux_2.index
new_train_proteins = protein_aux_3.merge(protein_aux_4, left_index=True, right_index=True, how="inner")
del protein_aux_1, protein_aux_2, protein_aux_3, protein_aux_4
new_train_proteins.reset_index(inplace=True)
print(f"New Train Proteins Dataframe shape has {new_train_proteins.shape[0]} rows and {new_train_proteins.shape[1]} columns")
display(new_train_proteins.head())
train_df = train_clinical_data.merge(new_train_proteins,
                                     on=["patient_id", "visit_id", "visit_month"],
                                     how="left")

New Train Proteins Dataframe shape has 1113 rows and 451 columns


,patient_id,visit_id,visit_month,UniProt_0,UniProt_1,UniProt_2,UniProt_3,UniProt_4,UniProt_5,UniProt_6,...,NPX_214,NPX_215,NPX_216,NPX_217,NPX_218,NPX_219,NPX_220,NPX_221,NPX_222,NPX_223
0,55,55_0,0,0,1,2,3,4,5,6,...,60912.6,408698.0,29758.8,23833.7,18953.5,NaN,NaN,NaN,NaN,NaN
1,55,55_12,12,0,1,2,3,4,5,6,...,114921.0,21860.1,61598.2,318553.0,65762.6,29193.4,28536.1,19290.9,NaN,NaN
2,55,55_36,36,0,1,2,3,4,5,6,...,303597.0,48188.4,109794.0,23930.6,70223.5,377550.0,74976.1,31732.6,22186.5,21717.1
3,55,55_6,6,0,1,2,3,4,5,6,...,369870.0,22935.2,17722.5,16642.7,NaN,NaN,NaN,NaN,NaN,NaN
4,942,942_12,12,0,1,2,3,4,5,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
class config:
    MAX_PROTEINS = 20
    PROFILE_REPORT = True

In [40]:
le_cols_1 = [col for col in train_df.columns if 'UniProt' in col][0:config.MAX_PROTEINS]
le_cols_2 = [col for col in train_df.columns if 'NPX' in col][0:config.MAX_PROTEINS]
TRAIN_COLUMNS = le_cols_1 + le_cols_2 
TARGET_COLUMNS = ["updrs_1", "updrs_2", "updrs_3", "updrs_4"]

# === Filla NA ===
train_df.drop(columns=["upd23b_clinical_state_on_medication"], inplace=True)
train_df.loc[:, train_df.columns.isin(TARGET_COLUMNS)] = train_df.loc[:, train_df.columns.isin(TARGET_COLUMNS)].fillna(0)
train_df.loc[:, train_df.columns.isin(le_cols_1)] = train_df.loc[:, train_df.columns.isin(le_cols_1)].fillna(-1)
train_df.loc[:, train_df.columns.isin(le_cols_2)] = train_df.loc[:, train_df.columns.isin(le_cols_2)].fillna(0)

# === Set Index ===
train_df.set_index(["visit_id", "patient_id", "visit_month"], inplace=True)
train_df.head()

,,,updrs_1,updrs_2,updrs_3,updrs_4,UniProt_0,UniProt_1,UniProt_2,UniProt_3,UniProt_4,UniProt_5,...,NPX_214,NPX_215,NPX_216,NPX_217,NPX_218,NPX_219,NPX_220,NPX_221,NPX_222,NPX_223
visit_id,patient_id,visit_month,,,,,,,,,,,,,,,,,,,,,
55_0,55,0,10.0,6.0,15.0,0.0,0.0,1.0,2.0,3.0,4.0,5.0,...,60912.6,408698.0,29758.8,23833.7,18953.5,NaN,NaN,NaN,NaN,NaN
55_3,55,3,10.0,7.0,25.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55_6,55,6,8.0,10.0,34.0,0.0,0.0,1.0,2.0,3.0,4.0,5.0,...,369870.0,22935.2,17722.5,16642.7,NaN,NaN,NaN,NaN,NaN,NaN
55_9,55,9,8.0,9.0,30.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55_12,55,12,10.0,10.0,41.0,0.0,0.0,1.0,2.0,3.0,4.0,5.0,...,114921.0,21860.1,61598.2,318553.0,65762.6,29193.4,28536.1,19290.9,NaN,NaN


In [41]:
from sklearn.model_selection import train_test_split

X = train_df.loc[:, train_df.columns.isin(TRAIN_COLUMNS)]
y = train_df.loc[:, train_df.columns.isin(TARGET_COLUMNS)]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
from sklearn.multioutput import MultiOutputRegressor

# create an xgboost regression model
xgb_reg = MultiOutputRegressor(
    xgb.XGBRegressor(n_estimators=300, max_depth=15, objective='reg:squarederror')
)
xgb_reg = xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test)

In [50]:
def process_predictions(y_pred, X_test):
    predictions = []

    for i in range(y_pred.shape[1]):  
        pred = pd.DataFrame(y_pred[:,i])
        pred.columns = ["rating"]
        pred["updrs"] = TARGET_COLUMNS[i]
        pred.index = X_test.index
        predictions.append(pred)

    predictions = pd.concat(predictions)
    predictions.reset_index(inplace=True)
    return predictions

predictions = process_predictions(y_pred, X_test)
predictions.head()

,visit_id,patient_id,visit_month,rating,updrs
0,64674_84,64674,84,7.721416,updrs_1
1,24820_18,24820,18,7.582389,updrs_1
2,56317_9,56317,9,7.582389,updrs_1
3,26210_42,26210,42,7.582389,updrs_1
4,49683_72,49683,72,7.582389,updrs_1


In [52]:
def process_test(test_clinical_data, test_proteins):
    # === Preprocess Protein DataFrame ===
    test_proteins.loc[:, "UniProt"] = test_proteins.loc[:, "UniProt"].fillna("None")
    test_proteins.loc[:, "NPX"] = test_proteins.loc[:, "NPX"].fillna(0)
    test_proteins['UniProt'] = le.fit_transform(test_proteins['UniProt'])
    protein_aux_1 = pd.DataFrame(test_proteins.groupby(by=["patient_id", "visit_id", "visit_month"])["UniProt"].apply(list))#.reset_index()
    protein_aux_2 = pd.DataFrame(test_proteins.groupby(by=["patient_id", "visit_id", "visit_month"])["NPX"].apply(list))#.reset_index()
    protein_aux_3 = pd.DataFrame(protein_aux_1.UniProt.values.tolist()).add_prefix('UniProt_')
    protein_aux_4 = pd.DataFrame(protein_aux_2.NPX.values.tolist()).add_prefix('NPX_')
    protein_aux_3.index = protein_aux_1.index
    protein_aux_4.index = protein_aux_2.index
    new_test_proteins = protein_aux_3.merge(protein_aux_4, left_index=True, right_index=True, how="inner")
    del protein_aux_1, protein_aux_2, protein_aux_3, protein_aux_4
    new_test_proteins.reset_index(inplace=True)
    
    # === Merge Clinical Data with Protein Data ===
    test_df = test_clinical_data.merge(new_test_proteins,
                                     on=["patient_id", "visit_id", "visit_month"],
                                     how="left")
    
    # === Select Label Encoder Columns ===
    le_cols_1 = [col for col in test_df.columns if 'UniProt' in col][0:config.MAX_PROTEINS]
    le_cols_2 = [col for col in test_df.columns if 'NPX' in col][0:config.MAX_PROTEINS]
    TRAIN_COLUMNS = le_cols_1 + le_cols_2 
    
    # === Fill NA ===
    test_df.loc[:, test_df.columns.isin(le_cols_1)] = test_df.loc[:, test_df.columns.isin(le_cols_1)].fillna(-1)
    test_df.loc[:, test_df.columns.isin(le_cols_2)] = test_df.loc[:, test_df.columns.isin(le_cols_2)].fillna(0)
    test_df.loc[:, test_df.columns.isin(TARGET_COLUMNS)] = test_df.loc[:, test_df.columns.isin(TARGET_COLUMNS)].fillna(0)
    
    # === Encode columns ===
    test_df.set_index(["visit_id", "patient_id", "visit_month"], inplace=True)
    X_test = test_df.loc[:, test_df.columns.isin(TRAIN_COLUMNS)]
    y_pred = xgb_reg.predict(X_test)
    predictions = process_predictions(y_pred, X_test)
    prediction_id = []

    auxs = [0,6,12,24]
    for index, group in predictions.groupby(["visit_id", "patient_id", "visit_month", "updrs"]):
        i = 0
        for index, row in group.iterrows():
            prediction_id.append(row["visit_id"] + "_" + row["updrs"] + "_plus_" + str(auxs[i]) + "_months")
            i += 1
    predictions["prediction_id"] = prediction_id
    print(f"Predictions shape has {predictions.shape[0]} rows and {predictions.shape[1]} columns")
    return predictions

In [54]:
test = pd.read_csv("example_test_files/test.csv")
print(f"Test Dataframe shape has {test.shape[0]} rows and {test.shape[1]} columns")
display(test.head())

test_proteins = pd.read_csv("example_test_files/test_proteins.csv")
print(f"Test Proteins shape has {test_proteins.shape[0]} rows and {test_proteins.shape[1]} columns")
display(test_proteins.head())

sample_submission = pd.read_csv("example_test_files/sample_submission.csv")
print(f"Sample Submission shape has {sample_submission.shape[0]} rows and {sample_submission.shape[1]} columns")
display(sample_submission.head())

Test Dataframe shape has 16 rows and 6 columns


,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
4,50423_0,0,50423,updrs_1,50423_0_updrs_1,0


Test Proteins shape has 453 rows and 6 columns


,visit_id,visit_month,patient_id,UniProt,NPX,group_key
0,50423_0,0,50423,O00391,33127.90,0
1,50423_0,0,50423,O00533,490742.00,0
2,50423_0,0,50423,O00584,43615.30,0
3,50423_0,0,50423,O14773,16486.60,0
4,50423_0,0,50423,O14791,2882.42,0


Sample Submission shape has 64 rows and 3 columns


,prediction_id,rating,group_key
0,3342_0_updrs_1_plus_0_months,0,0
1,3342_0_updrs_1_plus_6_months,0,0
2,3342_0_updrs_1_plus_12_months,0,0
3,3342_0_updrs_1_plus_24_months,0,0
4,3342_0_updrs_2_plus_0_months,0,0


import amp_pd_peptide
env = amp_pd_peptide.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    sample_prediction_df['rating'] = np.arange(len(sample_prediction))  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions